In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv("sql12.csv")

In [4]:
data.head(5)

,ID,projectname,classification,Abstract,label
0,0,sql12,DEFECT,/**\r\t * TODO: this is a bug in the code at t...,yes
1,1,sql12,DEFECT,// Cast to int required as Dimension::setSize(...,yes
2,2,sql12,DEFECT,/*\r * This won't work on Derby where no...,yes
3,3,sql12,DEFECT,// The following query produces a result that ...,yes
4,4,sql12,DEFECT,// We know this fails - Bug# 1700093,yes


In [5]:
labels_list=[]
for i, row in data.iterrows():
    labels_list.append(row['label'])
    
comments_list=[]
for i, row in data.iterrows():
    comments_list.append(row['Abstract'])

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
t=Tokenizer()
t.fit_on_texts(comments_list)
vocab_size = len(t.word_index) + 1

In [8]:
encoded_docs = t.texts_to_sequences(comments_list)
X,l= np.array(encoded_docs), np.array(labels_list)

In [9]:
max_length = 400
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)

In [10]:
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(l)
y=to_categorical(vec)

In [11]:
print(padded_docs.shape)
print(y.shape)

(7215, 400)
(7215, 2)


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
data = {}
data["X_train"] = X_train
data["X_test"]= X_test
data["y_train"] = y_train
data["y_test"] = y_test
data["tokenizer"] = t
data["int2label"] =  {0: "no", 1: "yes"}
data["label2int"] = {"no": 0, "yes": 1}
    

In [14]:
#data

In [43]:
'''indices = np.arange(padded_docs.shape[0])
np.random.shuffle(indices)
data = padded_docs[indices]
labels = y[indices]
nb_validation_samples = int(0.1 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]'''

'indices = np.arange(padded_docs.shape[0])\nnp.random.shuffle(indices)\ndata = padded_docs[indices]\nlabels = y[indices]\nnb_validation_samples = int(0.1 * data.shape[0])\n\nx_train = data[:-nb_validation_samples]\ny_train = labels[:-nb_validation_samples]\nx_val = data[-nb_validation_samples:]\ny_val = labels[-nb_validation_samples:]'

In [15]:
embeddings_index = {}
f = open('GoogleNews-vectors-negative300.bin', 'rb')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 4410354 word vectors.


In [16]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, encoding="ISO-8859-1", unicode_errors='ignore')
#model.save_word2vec_format('GoogleNews-vectors-negative300.txt', binary=False)

In [17]:
word2index = {token: token_index for token_index, token in enumerate(model.index2word)}

In [18]:
embedding_matrix = np.zeros((len(word2index) + 1, 300))

for word, i in word2index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [19]:
vocab = model.vocab.keys()
wordsInVocab = len(vocab)
print (wordsInVocab)

3000000


In [20]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from keras.initializers import Constant
from tensorflow.keras.models import Sequential,load_model

In [55]:
model = Sequential()
model.add(Embedding(len(word2index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=400,
                            trainable=False))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='softmax'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

history = model.fit(data["X_train"], data["y_train"],
                    batch_size=128,
                    epochs=5,
                    validation_data=(data["X_test"], data["y_test"]),
                    verbose=1)

loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('\n\n\nTest Accuracy: %f' % (acc*100))

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
def get_predictions(text):
    sequence = data["tokenizer"].texts_to_sequences([text])
    # pad the sequences
    sequence = pad_sequences(sequence, maxlen=SEQUENCE_LENGTH)
    # get the prediction
    prediction = model.predict(sequence)[0]
    return prediction, data["int2label"][np.argmax(prediction)]

In [ ]:
text = "// TODO fix the space above."
output_vector, prediction = get_predictions(text)
print("Output vector:", output_vector)
print("Prediction:", prediction)